In [ ]:
''' Main Notion Integration '''
import os
from notion_client import Client
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import csv
from helpers import * # not needed 
from dotenv import load_dotenv

load_dotenv()

NOTION_TOKEN = os.environ.get('NOTION_TOKEN')
DATABASE_ID = os.environ.get('APPLICATION_DATABASE_ID')

NUM_SENT_LAST_30_DAYS_FILE_NAME = 'num-sent-30-days'
NUM_SENT_LAST_30_DAYS_BLOCK_ID = '416890d79a53494daf530fe5c8ca1f10'
# https://www.notion.so/My-Applicatio-416890d79a53494daf530fe5c8ca1f10?pvs=4#40abebf968cb4d4e8d96db7e39bd30aa

In [ ]:
# Initialize the client
notion = Client(auth=NOTION_TOKEN)

# Query the database
database = notion.databases.query(DATABASE_ID).get('results')

In [ ]:
print("type(database): " + str(type(database)))
print("len(database): " + str(len(database)))
print(type(database[0]))
print(database[0].keys())
# print('=========')
# for k in database[0].keys():
#     print(f'{k}: {database[0].get(k)}')

In [ ]:
testtest = database[0]['properties'].keys()
print(type(testtest))
print(testtest)

In [ ]:
temptemp = database[0]['properties']['Date Applied']['date']['start']
print(type(temptemp))
print(temptemp)

In [ ]:
db_days_elapsed = []
db_statuses = []
db_date_applied = []

for d in database:
    db_days_elapsed.append(d['properties']['days elapsed']['formula']['number'])
    db_statuses.append(d['properties']['Status']['select']['name'])
    db_date_applied.append(d['properties']['Status']['select']['name'])


print(f"db_days_elapsed: {len(db_days_elapsed)}")
print(f"db_statuses: {len(db_statuses)}")
print(f"db_date_applied: {len(db_date_applied)}")

In [ ]:
# Generate a list of the last 30 days with today at the end of the list
end_date = datetime.today()
start_date = end_date - pd.Timedelta(days=29)

dates = pd.date_range(start=start_date, end=end_date, freq='D')
dates_list = dates.to_list()

print(dates_list)

In [ ]:
ct_ = 0
for entry in database:
    if ct_ >= 5:
        break
    else:
        dateVal = entry['properties']['Date Applied']['date']['start']
        print(type(dateVal))
        ct_ += 1

In [ ]:
''' Parse Database response into Pandas Dataframe '''
data = {
    'days_elapsed': [],
    'date_applied': [],
    'status': [],
    'state': [],
    'job_path': []
}

for entry in database:
    properties = entry['properties']

    data['days_elapsed'].append(properties['days elapsed']['formula']['number'])
    data['date_applied'].append(properties['Date Applied']['date']['start'])
    data['status'].append(properties['Status']['select']['name']) # 'Rejected', 
    data['state'].append(properties['state']['state']['name'])
    data['job_path'].append(properties['Status']['select']['name'])

In [ ]:
df = pd.DataFrame.from_dict(data)
df['date_applied'] = pd.to_datetime(df['date_applied'])
df = df.sort_values(by='days_elapsed')
df

In [ ]:
filtered_df = df[df['days_elapsed'] <= 31]

In [ ]:
# Current date
today = datetime.today()

# Generate a list of datetime objects for the last 30 days
last_30_days = [today - timedelta(days=i) for i in range(30, -1, -1)] # 31 Days Generated

x_vals = []
y_vals = []

for i in range(30, -1, -1):
    ct = 0
    last_date = last_30_days[i].date()

    for j, row in df.iterrows():
        app_date = row['date_applied'].date()
        if last_date == app_date:
            ct += 1
        elif last_date > app_date:
            break
    
    formatted_last_date = last_date.strftime('%m-%d-%Y')
    x_vals.append(formatted_last_date)
    y_vals.append(ct)

x_vals.reverse()
y_vals.reverse()

In [ ]:
# palette=sns.color_palette("light:#5A9", as_cmap=True)
fig, ax = plt.subplots(figsize=(12, 6))

# Set the background color of the entire figure
fig.patch.set_facecolor('lightgray')

# Set the background color of the axes (plot area)
ax.set_facecolor('darkgray')

ax.bar(x_vals, y_vals, width=0.95, color='MediumAquaMarine')

# Add plot labels
ax.set_title('# Applications Sent per Day [Last 30 Days]')
ax.set_xlabel(None)
ax.set_ylabel('#')
plt.xticks(rotation=45)
# plt.savefig('images/pca_scree_plot.png')
plt.show()

In [ ]:
print(df.dtypes['date_applied'])
print(type(last_30_days[0]))

In [ ]:
grouped